# Reinforcement Learning: Project 1

* Matheus Gustavo Alves Sasso - 158257
* Fabiano
* Lucas
* Rafael Cortez Sanchez - 094324

## Goal

### Definição do Problema

R: Dado um labirinto bidimensional e um agente capaz de navegar por esse labirinto, aprender o menor caminho entre dois pontos fornecidos. O labirinto também possui casas pareadas que transportam o agente de uma casa para a outra, adicionando complexidade ao problema. O código base utiliza a biblioteca do OpenAI Gym e está disponível [neste link](https://github.com/MattChanTK/gym-maze).

### Formulação MDP

R: O MDP é uma tupla $<S, P, A, R, \gamma>$ tal que:

$S = \{s^{i, j} | i, j \in \mathbb{Z};  0 \lt i \leq m;  0 \lt j \leq n\}$, em que $m$ e $n$ são as dimensões do labirinto em número inteiro de casas.

$P = \{P^{r, s} = \begin{cases}
                      1.0               & \text{se não há paredes entre os estados r e s}\\
                      0.0               & \text{caso contrário}
                  \end{cases} \}$

$A = \{\text{cima}, \text{baixo}, \text{esquerda}, \text{direita}\}$

$R = \{R^s = \begin{cases}
                      1.0                        & \text{caso s seja o estado final}\\
                      \frac{-0.1}{m \times n}    & \text{caso contrário}
             \end{cases} \}$

$\gamma = 0.99$

### O modelo de discretização adotado


R: O problema em si já é discretizado, uma vez que o labirinto é composto por um número inteiro de casas dispostas em um plano. Cada estado é definido por duas coordenadas $i, j \in \mathbb{Z}$, representando uma casa desse plano.

### Como o problema foi modelado 
**Avaliar de acordo com: custo computacional, optimalidade, influência da função de recompensa, tamanhos dos espaços de estados e de ações**


R: Considerando que as dimensões do mapa são $n$ e $m$, temos os seguintes tamanhos para os espaços de estados e ações:

$|S| = n \times m$

$|A| = 4$ (As quatro direções cardinais no plano)

O custo computacional de resolver a equação de Bellman por sistema linear de equações é, portanto:

$O(n^3 m^3)$

A solução ótima é uma política $\pi$ que minimiza o número de ações para chegar à casa final do labirinto a partir da casa inicial. Há uma recompensa negativa quando o agente se move para casas que não são a casa final, e uma recompensa positiva quando o agente atinge a casa final. A recompensa negativa é inversamente proporcional ao tamanho do labirinto, de forma que a penalidade é maior quando se navega em um labirinto pequeno.

### Especificidades e restrições da implementação

R: O agente aprende a navegar em um único labirinto: a solução não é generalizada. A interface gráfica da solução foi implementada usando a biblioteca pygame e, caso o programa não seja devidamente finalizado, pode ser necessário reiniciar o interpretador Python antes de reinciar a execução do programa.

## Ambiente do Problema

### The nature of your environment (episodic/not episodic, deterministic/stochastic)


R:

### What are your terminal states (when they exist)


R:

### How is your reward function defined

R:

### All parameters employed in your methods (discount factor, step size, etc.)


R:

### Enverionment Class Code

In [ ]:
## Code Here

## Problem Implementations Without Linear Function Aproximators

### Monte Carlo Control
* Initialize the value function to zero. 
* Use a time-varying scalar step-size of αt = 1/N(st, at)
* Use an  Epsilon−greedy exploration strategy with Epsilon_t = N0/(N0+N(st)), where N0 is a constant, N(s) is the number of times that state s has been visited, and N(s, a) is the number of times that action a has been selected
from state s. 
* You should define N0 to fit your problem. 
* Plot the optimal value function V(s) = maxaQ∗ (s, a).


Algorithim Description

R: 

Implementation

In [ ]:
## Code Here

Results (Graphs and tables)

R: 

Relationship between adopted parameters x solution performance.

R: 

Comparition With the traditional algorithm

R: 

### Q-learning (or some variation like DoubleQ-learning)
* Initialize the action-value function to zero. 
* Use a random selection of actions whenever you have a draw among actions.
* Use the same step-size and exploration schedules as MC Control. 
* Define the number of episodes and discount factor accordingly.

Algorithim Description

R:

Implementation

In [ ]:
## Code here

Results (Graphs and tables)

R: 

Relationship between adopted parameters x solution performance.

R: 

Comparition With the traditional algorithm

R: 

### SARSA(λ): 
* Initialize the action-value function to zero. 
* Use a random selection of actions whenever you have a draw among actions. 
* Use the same step-size and exploration schedules as MC Control.
* Use the same number of episodes as Q-learning. 
* Vary λ ∈ {0, 0.2, ..., 1}

Algorithim Description

R: 

Implementation

In [ ]:
## Code Here

Results (Graphs and tables)

R: 

Relationship between adopted parameters x solution performance.

R: 

Comparition With the traditional algorithm

R: 

## Problem Implementations With Linear Function Aproximators

### Monte Carlo Control

Algorithim Description

R: 

Implementation

In [ ]:
## Code Here

Results (Graphs and tables)

R: 

Relationship between adopted parameters x solution performance.

R: 

Comparition With the traditional algorithm

R: 

### Q-learning (or some variation like DoubleQ-learning)

Algorithim Description

R: 

Implementation

In [ ]:
## Code Here

Results (Graphs and tables)

R: 

Relationship between adopted parameters x solution performance.

R: 

Comparition With the traditional algorithm

R: 

### SARSA(λ)

Algorithim Description

R: 

Implementation

In [ ]:
## Code Here

## Evaluation and Discussion

### The advantages and disadvantages of bootstrapping in your problem.

R: 

### How the reward function influenced the quality of the solution. Was your group able to achieve the expected policy given the reward function defined?

R: 

### How function approximation influenced the results. What were the advantages and disadvantages of using it in your problem?


R: 

## Member Cotributions

**Matheus**

**Fabiano**

**Lucas**

**Raphael**